<a href="https://colab.research.google.com/github/mahdavipanah/IoT-Colab/blob/main/IoT_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Link to the paper https://www.sciencedirect.com/science/article/pii/S0959652621016267

In [ ]:
pip install dataframe-image

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import dataframe_image as dfi

%load_ext google.colab.data_table

from google.colab import data_table
data_table._DEFAULT_FORMATTERS[float] = lambda x: f"{x:.4f}"

In [125]:
#@title #Parameters

#@markdown Prior knowledge for the probability of the event occurrence:
PE = 0.7 #@param {type:"number"}

#@markdown Selection ratio in the list of nodes
S = 100 #@param {type:"integer"}
K = 10 #@param {type:"integer"}

#@markdown Number of trusted nodes:
TRUSTED_NODES_COUNT = 90 #@param {type:"integer"}

#@markdown Trusted nodes probability range (inclusive):
P_START = 0.5 #@param {type:"number"}
P_END = 0.9 #@param {type:"number"}


#@markdown Malicious nodes probability range (inclusive)
MALICIOUS_P_START = 0.3 #@param {type:"number"}
MALICIOUS_P_END = 0.39 #@param {type:"number"}

#@markdown Security level for calculating ideal value for Nash EQ:
S_STAR_DELTA = 2 #@param {type:"number"}

#@markdown Default trust value:
DEFAULT_TRUST = 0.5 #@param {type:"number"}

#@markdown Punishment and reward for trust of nodes based on Nash EQ:
TRUST_PUNISH_VALUE = 0.1 #@param {type:"number"}

#@markdown Maximum and minimum for trust values:
TRUST_MAX = 0.9 #@param {type:"number"}
TRUST_MIN = 0.1 #@param {type:"number"}


rng = np.random.default_rng()
# trusted nodes
trusted_nodes = rng.uniform(
  P_START,
  P_END,
  TRUSTED_NODES_COUNT
)
# malicious nodes 
malicious_nodes = rng.uniform(
  MALICIOUS_P_START,
  MALICIOUS_P_END,
  S - TRUSTED_NODES_COUNT,
)


malicious_indices = rng.choice(S, size=S - TRUSTED_NODES_COUNT, replace=False)

nodes = np.array([])
for i in np.arange(S):
  elem = None

  if i in malicious_indices:
    elem, malicious_nodes = malicious_nodes[0], np.delete(malicious_nodes, 0)
  else:
    elem, trusted_nodes = trusted_nodes[0], np.delete(trusted_nodes, 0)

  nodes = np.append(nodes, elem)

data_frame = pd.DataFrame(nodes, columns = ['probability'])
data_frame['trust'] = DEFAULT_TRUST

for n in np.arange(S//K):
  # indecies of the n-th E-Sensor
  sensor_indices = np.arange(n * K, n * K + K)
  
  for i in sensor_indices:
    i_prob = data_frame.at[i, 'probability']

    # indices of other nodes of the sensor
    others_indices = np.delete(
      sensor_indices,
      np.where(sensor_indices == i)
    )

    # average of other nodes of the sensor
    others_average = np.mean(
        data_frame.loc[others_indices, 'probability'].values
    )
    data_frame.at[i, 'average'] = others_average

    # cost1
    i_cost1 = (i_prob - others_average) ** 2
    data_frame.at[i, 'cost1'] = i_cost1
    
    # cost2 calculations
    consistant_t = 1
    non_consistent_t = 1
    
    for j in others_indices:
      j_prob = data_frame.at[j, 'probability']
      j_trust = data_frame.at[j, 'trust']

      condition = (j_prob - 0.5) * (PE - 0.5)
      
      if condition < 0:
        consistant_t *= j_trust
      else:
        non_consistent_t *= j_trust
    
    pi_et = ((PE * consistant_t) /
             (PE * consistant_t + (1 - PE) * non_consistent_t))

    i_cost2 = (i_prob - pi_et) ** 2
    data_frame.at[i, 'cost2'] = i_cost2

    # utility
    data_frame.at[i, 'utility'] = i_cost1 + i_cost2
  
    i_s_star = (others_average + pi_et) / 2
    data_frame.at[i, 's*'] = i_s_star

  s_star_average = np.mean (
      data_frame.loc[sensor_indices, 's*'].values
  )
  
  s_star_change = (1 / S_STAR_DELTA) * s_star_average

  e_node_stake = 0

  for i in sensor_indices:
    i_prob = data_frame.at[i, 'probability']
    i_trust = data_frame.at[i, 'trust']

    if (i_prob <= s_star_average + s_star_change
        and i_prob >= s_star_average - s_star_change):
      
        i_new_trust = i_trust + TRUST_PUNISH_VALUE
        if (i_new_trust > TRUST_MAX):
          i_new_trust = i_trust
        
        e_node_stake += 1
    else:
      i_new_trust = i_trust - TRUST_PUNISH_VALUE
      if (i_new_trust < TRUST_MIN):
          i_new_trust = i_trust
    
    data_frame.at[i, 'new trust'] = i_new_trust
  
  data_frame.at[sensor_indices[0], 'stake'] = e_node_stake
    
data_frame

,probability,trust,average,cost1,cost2,utility,s*,new trust,stake
0,0.59358627459207990995,0.50000000000000000000,0.65999252922193540893,0.00440979065396520514,0.16247081338684421503,0.16688060404080942711,0.82832774403254727513,0.59999999999999997780,9.00000000000000000000
1,0.73855523558346258817,0.50000000000000000000,0.64388486688955959281,0.00896247870863952842,0.06661959680630409608,0.07558207551494362797,0.82027391286635942258,0.59999999999999997780,nan
2,0.75567635106423192148,0.50000000000000000000,0.64198252072502959287,0.01292628705719932325,0.05807454512879450803,0.07100083218599383128,0.81932273978409431159,0.59999999999999997780,nan
3,0.33925047241874567439,0.50000000000000000000,0.68825206279675021914,0.12180211008637648051,0.43548539732765156884,0.55728750741402799385,0.84370785464900233119,0.40000000000000002220,nan
4,0.75631554680989054873,0.50000000000000000000,0.64191149897551191827,0.01308828616089079307,0.05776687847108978591,0.07085516463198057724,0.81928722890933558531,0.59999999999999997780,nan
...,...,...,...,...,...,...,...,...,...
95,0.63645889096719820976,0.50000000000000000000,0.65443940592935001543,0.00032329891830416494,0.13155473968701950072,0.13187803860532365530,0.82680152621530222934,0.59999999999999997780,nan
96,0.62513842226754490206,0.50000000000000000000,0.65569723578486716065,0.00093384108358647762,0.13989486836307671314,0.14082870944666320279,0.82743044114306085746,0.59999999999999997780,nan
97,0.67025333523197960783,0.50000000000000000000,0.65068446767770771189,0.00038294057735663534,0.10818199285925125808,0.10856493343660789108,0.82492405708948113308,0.59999999999999997780,nan
98,0.56186644895820969658,0.50000000000000000000,0.66272745504145980089,0.01017294254812541390,0.19122883897900069816,0.20140178152712612247,0.83094555077135712207,0.59999999999999997780,nan
